In [ ]:
from lusidtools.jupyter_tools import toggle_code

"""Look-through Valuation with Multiple Levels

The following notebook details how to compute PVs across different levels of look-through for 
both securitised funds and Index Futures 

Attributes
----------
valuation
transactions
instruments
recipes
futures
securitised portfolios
"""

toggle_code("Hide docstring")

# Look-through Valuation and Reporting


## Table of contents

- 1. [Overview](#1.-Overview)
- 2. [Setup](#2.-Setup)
- 3. [Load data](#3.-Load-Data)
   * [3.1 Portfolios](#3.1-Portfolios)
   * [3.2 Instruments](#3.2-Instruments)
   * [3.3 Index Constituents](#3.3-Index-Constituents)
   * [3.4 Transactions](#3.4-Transactions)   
   * [3.5 Quotes](#3.5-Quotes)
- 4. [Run Valuations](#4.-Run-Valuations)
    * [4.1 Create Recipes](#4.1-Create-Recipes)
    * [4.2 Create Valuation Function](#4.2-Create-Valuation-Function)
    * [4.3 Top Level Valuation](#4.3-Top-Level-Valuation)
    * [4.4 Fund Level Look-through Valuation](#4.4-Fund-Level-Look-through-Valuation)
    * [4.5 Full Look-through Valuation](#4.5-Full-Look-through-Valuation)

## 1. Overview

In this Notebook, we present some of the look-through valuation capabilities available in LUSID across a multi-level security and fund structure. We look at these in the context of several securitised portfolios as well as an Equity Index Future. 

Our securitised portfolio hiearchy is constructed of two levels including a 'Global Equity' portfolio at the top level, and three sector specific securitised portfolios at the second level. 

The three sector specific portfolios focus on financials, energy, and technology. Each contains direct equity holdings in individual stocks, while the 'Global Equity Technology' portfolio holds an additional position in an Equity Index Future (the NYSE FANG+).

In the proceeding sections of this Notebook, we'll load all of the requisite data required to model these holdings while performing valuations for January 7th, 2022. 

Importantly, we'll walk through how we can drill down to the underlying positions and index constituents of our holdings in order to give us a clear view of our exposures. 

At a high level, the structure of our example looks as follows:

![Init](img/LookthroughMultiExample.PNG)

## 2. Setup

We first initialize our various Python libraries, objects, and datasets required to construct our examples:

In [2]:
# Import system packages

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
import json
import pytz
from datetime import datetime
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from flatten_json import flatten

import os
import pandas as pd
import math

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

#Load LUSID API Components
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
reference_portfolio_api = api_factory.build(lusid.api.ReferencePortfolioApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lusid.api.AggregationApi)

# Set Global Scope
scope = "ibor"

# Transaction Portfolios
transaction_portfolios = ["GlobalEquity", "GlobalEquityEnergy", "GlobalEquityFinancials", "GlobalEquityTechnology"]
fund_portfolios = transaction_portfolios[:3]

# Reference Portfolios
reference_portfolios = ["NYSEFANGPIndex"]

# Load Requisite Data
transaction_data = pd.read_excel("data/lookthrough_data.xlsx", sheet_name="transactions")
index_data = pd.read_excel("data/lookthrough_data.xlsx", sheet_name="index_weights")
price_data = pd.read_excel("data/lookthrough_data.xlsx", sheet_name="market_prices")
instrument_data = pd.read_excel("data/lookthrough_data.xlsx", sheet_name="instruments")

## 3. Load Data

The majority of our data will be loaded from 'lookthrough_data.xlsx'. This includes our transactions, the majority of our instruments, market quotes, and the index constituent data of our Equity Index Future.

### 3.1 Portfolios 

We first start by constructing our portfolios. This includes: 

1. A single top level LUSID Transacion Portfolio representing the Global Equity portfolio.
1. Three Transaction Portfolios covering the energy, financials, and technology sectors. These will ultimately be securitised and referred to within a set of instrument definitions.
3. A Reference Portfolio containing an equity index (NYSE FANG+) which will be used to model the underlying constituents of a corresponding Equity Index Future.

##### 3.1.1 Transaction Portfolios

In [ ]:
# Create our Transaction Portfolios
def load_txn_portfolio(portfolio_code):
    try:
        transaction_portfolio_api.create_portfolio(
            scope=scope,
            create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
                display_name=portfolio_code,
                code=portfolio_code,
                base_currency="USD",
                created="2022-01-01",
            ),
        )
        print("Portfolio: " + portfolio_code + " loaded!")

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

for portfolio in transaction_portfolios:
    load_txn_portfolio(portfolio)

#### 3.1.2 Reference Portfolio

In [ ]:
# Create a Reference Portfolio for our index future
def load_ref_portfolio(portfolio_code):
    try:
        response = reference_portfolio_api.create_reference_portfolio(
            scope=scope,
            create_reference_portfolio_request=models.CreateReferencePortfolioRequest(
                display_name=portfolio,
                base_currency="USD",
                code=portfolio, 
                created="2022-01-01"
            ),
        )

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

for portfolio in reference_portfolios:
    load_ref_portfolio(portfolio)

### 3.2 Instruments

#### 3.2.1 Instrument Properties
Next, we'll define the instrument properties for which we'll aggregate on. These include:

- Sector
- Industry
- Region

In [ ]:
# Create instrument properties for use in a valuation aggregation
properties = [
    ("Sector", "string"),
    ("Industry", "string"),
    ("Region", "string") 
]

for property_code, dtype in properties:
    try:
        properties_api.create_property_definition(
            create_property_definition_request=models.CreatePropertyDefinitionRequest(
                domain="Instrument",
                scope=scope,
                code=property_code,
                display_name=property_code,
                data_type_id=models.ResourceId(code=dtype, scope="system"),
            )
        )
    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

#### 3.2.2 Create a Lookthrough Instrument
For our first instrument, we'll securitise an equity index via a LUSID 'SimpleInstrument'. This will allow us to refer to our index within a 'Future' instrument definition.

In [ ]:
equity = models.Equity(
    instrument_type="Equity",
    dom_ccy="USD",
)

equity_definition = models.InstrumentDefinition(
    name="ref_NYSEFANGInst",
    identifiers={"ClientInternal": models.InstrumentIdValue("ref_NYSEFANGInst")},
    definition=equity,
    look_through_portfolio_id=models.ResourceId(scope=scope, code="NYSEFANGPIndex")
)

upsert_request = {"ref_NYSEFANGInst": equity_definition}
upsert_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    request_body=upsert_request
)

equity_lookthroughinst_luid = upsert_response.values["ref_NYSEFANGInst"].lusid_instrument_id
print(equity_lookthroughinst_luid)

#### 3.2.3 Equities
Next, we load our publicly listed equity instruments as type 'Equity'.

In [7]:
instrument_data

,name,client_internal,ticker,currency,sector,industry,region,lookthrough_id
0,Baidu Inc,eq_us_BIDU,BIDU,USD,Communications,Internet and Media,China,NaN
1,Apple Inc,eq_us_AAPL,AAPL,USD,Technology,Consumer Electronics,United States,NaN
2,Alibaba Group,eq_us_BABA,BABA,USD,Consumer,Online Retail,China,NaN
3,Alphabet Inc,eq_us_GOOGL,GOOGL,USD,Communications,Internet and Media,United States,NaN
4,Microsoft Corp,eq_us_MSFT,MSFT,USD,Technology,Software,United States,NaN
5,Amazon,eq_us_AMZN,AMZN,USD,Consumer,Online Retail,United States,NaN
6,Tesla Inc,eq_us_TSLA,TSLA,USD,Consumer,Automotive,United States,NaN
7,NVIDIA Corp,eq_us_NVDA,NVDA,USD,Technology,Semiconductors,United States,NaN
8,Netflix,eq_us_NFLX,NFLX,USD,Communications,Entertainment,United States,NaN
9,Meta Platforms Inc,eq_us_FB,FB,USD,Communications,Social Media,United States,NaN


In [ ]:
# Load publicly listed equities
def load_equity(data):
          
    client_internal = "Instrument/default/ClientInternal"
    
    equity = models.Equity(
        instrument_type="Equity",
        dom_ccy=data["currency"],
    )

    equity_definition = models.InstrumentDefinition(
        name=data["name"],
        identifiers={"ClientInternal": models.InstrumentIdValue(data["client_internal"]),
                     "Ticker": models.InstrumentIdValue(data["ticker"])
                    },
        definition=equity,
        properties=[
                models.ModelProperty(
                key=f"Instrument/{scope}/Sector",
                value=models.PropertyValue(label_value=row['sector']),
            ),
                models.ModelProperty(
                key=f"Instrument/{scope}/Industry",
                value=models.PropertyValue(label_value=row['industry']),
            ),
                models.ModelProperty(
                key=f"Instrument/{scope}/Region",
                value=models.PropertyValue(label_value=row['region']),
            ),
        ],        
    )

    # upsert the instrument
    upsert_request = {client_internal: equity_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    equity_luid = upsert_response.values[client_internal].lusid_instrument_id

for index, row in instrument_data.iterrows():
    
    if not isinstance(row["lookthrough_id"], float):
        continue
        
    load_equity(row)

print ("Instruments Upserted!")

#### 3.2.4 Equity Index Future
Our equity index future will be modelled as a LUSID 'Future' instrument. Importantly, it will refer to the ref_NYSEFANGInst 'SimpleInstrument' via a 'ReferenceInstrument' underlying reference. We do this by passing in the ref_NYSEFANGInst's corresponding LusidInstrumentId.

In [9]:
# Function definition to create a future
def create_futures_contract(
        dom_ccy,
        contract_code,
        contract_month,
        contract_size,
        convention,
        country_id,
        fut_name,
        exchange_code,
        exchange_name,
        ticker_step,
        unit_value,
        ref_spot_price,
        start_date,
        maturity_date,
        fut_identifier,
):
    ctc = models.FuturesContractDetails(
        dom_ccy=dom_ccy,
        contract_code=contract_code,
        contract_month=contract_month,
        contract_size=contract_size,
        convention=convention,
        country=country_id,
        description=fut_name,
        exchange_code=exchange_code,
        exchange_name=exchange_name,
        ticker_step=ticker_step,
        unit_value=unit_value,
    )
     
    futuredef = models.Future(
        start_date=start_date,
        maturity_date=maturity_date,
        identifiers={},
        contract_details=ctc,
        contracts=1,
        ref_spot_price=ref_spot_price,
        
        underlying=lusid.models.ReferenceInstrument(
            instrument_type="ReferenceInstrument",
                instrument_id=equity_lookthroughinst_luid,
                instrument_id_type="LusidInstrumentId",
                scope="default"
            ),

        instrument_type="Future",
    )
    # persist the instrument
    futureDefinition = models.InstrumentDefinition(
        name=fut_name,
        identifiers={"ClientInternal": models.InstrumentIdValue(fut_identifier)},     
        definition=futuredef,
        properties=[
                models.ModelProperty(
                key=f"Instrument/{scope}/sector",
                value=models.PropertyValue(label_value="Technology"),
            ),
                models.ModelProperty(
                key=f"Instrument/{scope}/industry",
                value=models.PropertyValue(label_value="Multi"),
            ),
                models.ModelProperty(
                key=f"Instrument/{scope}/region",
                value=models.PropertyValue(label_value="Global"),
            ),
        ],             
    )
    batchUpsertRequest = {fut_identifier: futureDefinition}
    upsertResponse = instruments_api.upsert_instruments(request_body=batchUpsertRequest)
    futLuid = upsertResponse.values[fut_identifier].lusid_instrument_id
    print("FutureLuid: " + futLuid)
    print("RefInstLuid: " + equity_lookthroughinst_luid)

In [ ]:
#NYSE FANG+ Future Definition
start_date = datetime(2021, 6, 21, tzinfo=pytz.utc)
effectiveAt = datetime.today().replace(tzinfo=pytz.utc)
maturity_date = datetime(2022, 6, 17, tzinfo=pytz.utc)

dom_ccy = "USD"
contract_code = "FNG" 
contract_month = "M"
contract_size = 5
convention = "ActualActual"
country_id= "US"
fut_name = "Micro NYSE FANG+ Index Future Mar22"
exchange_code = "ICE"
exchange_name ="ICE Exchange"
ticker_step = 0.2
unit_value = 1
ref_spot_price_val = None
identifier = "fut_NYSEFANGP001"

# Create Futures Contract function
create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    ticker_step,
    unit_value,
    ref_spot_price_val,
    start_date,
    maturity_date,
    identifier,
)

#### 3.2.5 Funds (Securitised Transaction Portfolios)
Finally, we must securitise our industry specific Transaction Portfolios by creating a set of instruments which refer to them via the 'look_through_portfolio_id' field.

In [ ]:
# Securitise our transaction portfolios
for index, row in instrument_data.iterrows():

    if isinstance(row["lookthrough_id"], float):
        continue
        
    simple_instrument = models.SimpleInstrument(
        instrument_type="SimpleInstrument",
        dom_ccy=row["currency"],
        asset_class="Equities",
        simple_instrument_type="Fund"
    )

    print(row["client_internal"])
    response = instruments_api.upsert_instruments(
        request_body={
            "upsert_instrument": models.InstrumentDefinition(
                name=row["name"],
                definition=simple_instrument,
                identifiers={
                    "ClientInternal": models.InstrumentIdValue(value=row["client_internal"])
                },
                look_through_portfolio_id=models.ResourceId(
                    scope=scope, code=row["lookthrough_id"]
                ),
                properties=[
                        models.ModelProperty(
                        key=f"Instrument/{scope}/Sector",
                        value=models.PropertyValue(label_value=row['sector']),
                    ),
                        models.ModelProperty(
                        key=f"Instrument/{scope}/Industry",
                        value=models.PropertyValue(label_value=row['industry']),
                    ),
                        models.ModelProperty(
                        key=f"Instrument/{scope}/Region",
                        value=models.PropertyValue(label_value=row['region']),
                    ),
                ],
            )
        }
    )
    
print ("Securitised Funds Upserted!")   

### 3.3 Index Constituents

Next, we want to load in the constituent data of the NYSE FANG+ Index

In [ ]:
# Initialise a list to hold our constituents
constituents = []

# Iterate over instrument unvierse to add each constituent to our list
for _, row in index_data.iterrows():
    constituents.append(
        models.ReferencePortfolioConstituentRequest(
        instrument_identifiers={
            "Instrument/default/Ticker": row["ticker"]
        },
        weight=row["weight"],
        currency="USD",
        )
    )

# Create our request to add our constituents
constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
    effective_from="2022-01-07T00:00:00Z",
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=1,
    constituents=constituents,
)

# Call LUSID to upsert our constituents into our reference portfolio
response = api_factory.build(
    lusid.api.ReferencePortfolioApi
).upsert_reference_portfolio_constituents(
     scope=scope,
     code="NYSEFANGPIndex",
    upsert_reference_portfolio_constituents_request=constituents_request,
)

print(f"Constituents Upserted")

### 3.4 Transactions
To construct our holdings, we load in a set of transaction data across our four portfolios

In [13]:
# Load Transactions
for index, row in transaction_data.iterrows():

    primary_instrument_identifier = { "Instrument/default/ClientInternal": row["client_internal"] }
    
    if isinstance(row["client_internal"], float):
        primary_instrument_identifier = { "Instrument/default/Currency": row["cash_id"] }

    upsert_transactions = transaction_portfolio_api.upsert_transactions(
        scope=scope,
        code=row['portfolio'],
        transaction_request=[
            models.TransactionRequest(
                transaction_id=row["txn_id"],
                type=row["txn_type"],
                instrument_identifiers=primary_instrument_identifier,
                transaction_date=row["trade_date"],
                settlement_date=row["settle_date"],
                units=row["quantity"],
                transaction_price=models.TransactionPrice(
                    price=row["txn_price"], type="Price"
                ),
                total_consideration=models.CurrencyAndAmount(
                    amount=row["total_consideration"], currency=row["currency"]
                ),
            )
        ],
    )

### 3.5 Quotes
We now load in the relavent market prices for our valuation date of Jan 7th, 2022

In [14]:
# Display Futures Prices
price_data

,date,id,id_type,price,currency
0,2022-01-07T00:00:00Z,eq_us_BIDU,ClientInternal,146.00,USD
1,2022-01-07T00:00:00Z,eq_us_AAPL,ClientInternal,180.00,USD
2,2022-01-07T00:00:00Z,eq_us_BABA,ClientInternal,120.00,USD
3,2022-01-07T00:00:00Z,eq_us_GOOGL,ClientInternal,"2,888.00",USD
4,2022-01-07T00:00:00Z,eq_us_MSFT,ClientInternal,329.00,USD
5,2022-01-07T00:00:00Z,eq_us_AMZN,ClientInternal,"3,350.00",USD
6,2022-01-07T00:00:00Z,eq_us_TSLA,ClientInternal,"1,150.00",USD
7,2022-01-07T00:00:00Z,eq_us_NVDA,ClientInternal,293.00,USD
8,2022-01-07T00:00:00Z,eq_us_NFLX,ClientInternal,591.00,USD
9,2022-01-07T00:00:00Z,eq_us_FB,ClientInternal,337.00,USD


In [15]:
for index, row in price_data.iterrows():

    instrument_quotes = {
        "upsert_request": models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=row["id"],
                    instrument_id_type=row["id_type"],
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=row["date"],
            ),
            metric_value=models.MetricValue(value=row["price"], unit=row["currency"]),
        )
    }

    # Upsert the quotes into LUSID
    response = quotes_api.upsert_quotes(
        scope=scope, request_body=instrument_quotes
    )

## 4. Run Valuations

Now that the requisite data is in place to model our portfolios and holdings, we want to start running valuations. In order to do this, we'll create three recipes:

- DirectHoldingsValuationRecipe - The top level valuation recipe which will value all directly held positions with no look-through
- FundLookthroughValuationRecipe - The fund level valuation recipe will look through all securitised portfolios.
- FullLookthroughValuationRecipe - The full look-through valuation recipe will expand all securitised portfolios and index futures.

Importantly, we have two model rules which control look-through for SimpleInstruments and Futures. The first model rule specifies the use of an 'InlinedIndex' model which expands each SimpleInstrument into it's underlying holdings. Specifying the IndexModelOptions under the model_options field then tells LUSID how to scale the underlying portfolio PVs for lookthrough.

In our second model rule, we again specify an IndexModelOptions object to tell LUSID how we want to expand the Index Future during valuation. If we did not specify this, the Index Future would not expand to its underlying constituents.

### 4.1 Create Recipes

In [16]:
# Create three recipes for varying levels of look-through
def UpsertRecipe(recipe_code):
    
    model_rules=[]
        
    if recipe_code == "FundLookthroughValuationRecipe" or recipe_code == "FullLookthroughValuationRecipe":
        model_rules.append(models.VendorModelRule(
                supplier="Lusid",
                model_name="InlinedIndex",
                instrument_type="SimpleInstrument",
                model_options=models.IndexModelOptions(portfolio_scaling="AbsoluteSum", model_options_type="IndexModelOptions"),
                )
        )        

    if recipe_code == "FullLookthroughValuationRecipe":
        model_rules.append(models.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Future",
                model_options=models.IndexModelOptions(portfolio_scaling="AbsoluteSum", model_options_type="IndexModelOptions"),
            )
        )
    
    configuration_recipe = models.ConfigurationRecipe(
        scope=scope,
        code=recipe_code,
        market=models.MarketContext(
            market_rules=[
                models.MarketDataKeyRule(
                    key="Equity.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field="mid",
                    quote_interval="5D",
                )
            ],
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="ClientInternal",
                default_scope=scope,
                attempt_to_infer_missing_fx=True             
            ),
        ),
        pricing=models.PricingContext(
            model_rules=model_rules
        ),    
    )

    upsert_configuration_recipe_response = (
        configuration_recipe_api.upsert_configuration_recipe(
            upsert_recipe_request=models.UpsertRecipeRequest(
                configuration_recipe=configuration_recipe
            )
        )
    )
    
UpsertRecipe("DirectHoldingsValuationRecipe")
UpsertRecipe("FundLookthroughValuationRecipe")
UpsertRecipe("FullLookthroughValuationRecipe")

### 4.2 Create Valuation Function

We'll create a valuation function that allows us to aggregate on an instrument property. In our example, this will either be by industry, sector, or region.

In [17]:
# Create a valuation function allowing users to aggregate based on different instrument properies
# as well as by specifying the look-through recipe
def get_daily_val(date, portfolio_code, aggregate_field, recipe_code):
    
    group_by = []
    metricsList = []
    columnsToRename = {}
    
    if aggregate_field:
        metricsList.extend([
            models.AggregateSpec(f"Instrument/{scope}/{aggregate_field}", "Value"),
            models.AggregateSpec(f"Valuation/PvInReportCcy", "Sum"),        
            models.AggregateSpec(f"Valuation/ExposureInReportCcy", "Sum"),             
            models.AggregateSpec(f"Holding/default/Units", "Value"),              
        ])
        group_by.append(f"Instrument/{scope}/{aggregate_field}")
        columnsToRename={
            f"Sum(Valuation/PvInReportCcy)": "PV (Reporting Ccy)",
            f"Sum(Valuation/ExposureInReportCcy)": "Exposure (Reporting Ccy)"
        }
        
    else:
        metricsList.extend([
            models.AggregateSpec(f"Instrument/default/Name", "Value"),
            models.AggregateSpec(f"Instrument/{scope}/Sector", "Value"), 
            models.AggregateSpec(f"Instrument/{scope}/Industry", "Value"),
            models.AggregateSpec(f"Instrument/{scope}/Region", "Value"),             
            models.AggregateSpec(f"Valuation/PvInReportCcy", "Value"),        
            models.AggregateSpec(f"Valuation/ExposureInReportCcy", "Value"),   
            models.AggregateSpec(f"Holding/default/FundLineage", "Value"),
            models.AggregateSpec(f"Holding/default/Units", "Value"),             
        ])
        columnsToRename={
            f"Valuation/PvInReportCcy": "PV (Reporting Ccy)",
            f"Valuation/ExposureInReportCcy": "Exposure (Reporting Ccy)",
            f"Holding/default/FundLineage": "Fund Lineage",
        }
          
    # Build and run valuation request
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(scope=scope, code=recipe_code),
        metrics=metricsList,
        group_by=group_by,
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data
    vals_df = pd.DataFrame(val_data)
    
    if aggregate_field:
        columnsToRename[f"Instrument/{scope}/{aggregate_field}"] = f"{aggregate_field}"
    else:
        columnsToRename[f"Instrument/default/Name"] = "Instrument Name"
        columnsToRename[f"Instrument/{scope}/Sector"] = "Sector"
        columnsToRename[f"Instrument/{scope}/Industry"] = "Industry"
        columnsToRename[f"Instrument/{scope}/Region"] = "Region"

    vals_df.rename(
        columns=columnsToRename,
        inplace=True,
    )

    vals_df.drop('Holding/default/Units', axis=1, inplace=True)

    return vals_df

### 4.3 Top Level Valuation

We first look at a top-level valuation report unaggregated

In [18]:
get_daily_val("2022-01-07T00:00:00Z", "GlobalEquity", "", "DirectHoldingsValuationRecipe")

,Instrument Name,Sector,Industry,Region,PV (Reporting Ccy),Exposure (Reporting Ccy),Fund Lineage
0,GlobalEquityTechnology Fund,Technology,Multi,United States,"6,000.00","6,000.00",GlobalEquity
1,GlobalEquityEnergy Fund,Energy,Multi,Global,"5,750.00","5,750.00",GlobalEquity
2,GlobalEquityFinancials Fund,Financials,Multi,Global,"4,900.00","4,900.00",GlobalEquity
3,USD,None,None,None,"3,525.00","3,525.00",GlobalEquity


We then decide to aggregate on Sector

In [19]:
get_daily_val("2022-01-07T00:00:00Z", "GlobalEquity", "Sector", "DirectHoldingsValuationRecipe")

,Sector,PV (Reporting Ccy),Exposure (Reporting Ccy)
0,Technology,"6,000.00","6,000.00"
1,Energy,"5,750.00","5,750.00"
2,Financials,"4,900.00","4,900.00"
3,None,"3,525.00","3,525.00"


### 4.4 Fund Level Look-through Valuation

Next, we want to value our portfolio by drilling through to each securitised portfolio's underlying constituent position. We do this by using the 'FundLookthroughValuationRecipe' recipe.

In [20]:
get_daily_val("2022-01-07T00:00:00Z", "GlobalEquity", "", "FundLookthroughValuationRecipe")

,Instrument Name,Sector,Industry,Region,PV (Reporting Ccy),Exposure (Reporting Ccy),Fund Lineage
0,Intel Corporation,Technology,Semiconductors,United States,159.01,159.01,GlobalEquityTechnology
1,Oracle Corporation,Technology,Software,United States,66.75,66.75,GlobalEquityTechnology
2,"Uber Technologies, Inc",Technology,Software,United States,49.50,49.50,GlobalEquityTechnology
3,Micro NYSE FANG+ Index Future Mar22,None,None,None,"5,602.71","5,602.71",GlobalEquityTechnology
4,USD,None,None,None,122.03,122.03,GlobalEquityTechnology
5,BP plc,Energy,Oil and Gas,UK,"1,191.53","1,191.53",GlobalEquityEnergy
6,Exxon Mobil Corporation,Energy,Oil and Gas,United States,406.76,406.76,GlobalEquityEnergy
7,Chevron Corporation,Energy,Oil and Gas,United States,"2,982.93","2,982.93",GlobalEquityEnergy
8,Suncor Energy,Energy,Oil and Gas,Canada,214.54,214.54,GlobalEquityEnergy
9,USD,None,None,None,"1,167.74","1,167.74",GlobalEquityEnergy


We then aggregate by Industry

In [21]:
get_daily_val("2022-01-07T00:00:00Z", "GlobalEquity", "Industry", "FundLookthroughValuationRecipe")

,Industry,PV (Reporting Ccy),Exposure (Reporting Ccy)
0,Semiconductors,159.01,159.01
1,Software,116.25,116.25
2,None,"11,706.66","11,706.66"
3,Oil and Gas,"4,795.76","4,795.76"
4,Banking,"3,397.32","3,397.32"


It's important to note that LUSID performs look-through to the lowest level of granularity possible. In our example, we have a two level fund hierachy. If however this hierachy was three or more levels, LUSID would drill down to the 'leaf level' positions of this hierachy (i.e. where positions have no further lookthrough data constructed).

### 4.5 Full Look-through Valuation

Lastly, we want to value our portfolio by drilling down at both the securitised fund level, as well as at the Index Future constituent level.

In [22]:
get_daily_val("2022-01-07T00:00:00Z", "GlobalEquity", "", "FullLookthroughValuationRecipe")

,Instrument Name,Sector,Industry,Region,PV (Reporting Ccy),Exposure (Reporting Ccy),Fund Lineage
0,Intel Corporation,Technology,Semiconductors,United States,159.01,159.01,GlobalEquityTechnology
1,Oracle Corporation,Technology,Software,United States,66.75,66.75,GlobalEquityTechnology
2,"Uber Technologies, Inc",Technology,Software,United States,49.50,49.50,GlobalEquityTechnology
3,Baidu Inc,Communications,Internet and Media,China,560.27,560.27,GlobalEquityTechnology/NYSEFANGPIndex
4,Apple Inc,Technology,Consumer Electronics,United States,560.27,560.27,GlobalEquityTechnology/NYSEFANGPIndex
5,Alibaba Group,Consumer,Online Retail,China,560.27,560.27,GlobalEquityTechnology/NYSEFANGPIndex
6,Alphabet Inc,Communications,Internet and Media,United States,560.27,560.27,GlobalEquityTechnology/NYSEFANGPIndex
7,Microsoft Corp,Technology,Software,United States,560.27,560.27,GlobalEquityTechnology/NYSEFANGPIndex
8,Amazon,Consumer,Online Retail,United States,560.27,560.27,GlobalEquityTechnology/NYSEFANGPIndex
9,Tesla Inc,Consumer,Automotive,United States,560.27,560.27,GlobalEquityTechnology/NYSEFANGPIndex


We'll now aggregate on region. You'll notice the two Chinese based equities held in the Index Future (Baidu and Alibaba) add to the fund's China exposure.

In [23]:
get_daily_val("2022-01-07T00:00:00Z", "GlobalEquity", "Region", "FullLookthroughValuationRecipe")

,Region,PV (Reporting Ccy),Exposure (Reporting Ccy)
0,United States,"8,866.84","8,866.84"
1,China,"1,120.54","1,120.54"
2,None,"6,103.95","6,103.95"
3,UK,"1,210.42","1,210.42"
4,Canada,"2,873.26","2,873.26"
